In [10]:
tf_model_path = './inception_v1_2016_08_28_frozen.pb'

with open(tf_model_path, 'rb') as f:
    serialized = f.read()
    
# .pbファイル読み込み
tf_model_path = './stylize_quantized.pb'

with open(tf_model_path, 'rb') as f:
    serialized = f.read()

In [11]:
import tensorflow as tf

tf.reset_default_graph()
gdef = tf.GraphDef() 
gdef.ParseFromString(serialized)

563897

In [7]:
# ファイルからグラフ定義を取得
import tensorflow as tf

tf.reset_default_graph()
gdef = tf.GraphDef()
gdef.ParseFromString(serialized)

26735789

In [12]:
# 入力テンソルの名前を取得する
with tf.Graph().as_default() as g:
    tf.import_graph_def(gdef, name='')
    ops = g.get_operations()
    
    # プレースホルダの出力名を調べる
    N = len(ops)
    for i in range(N):
        if ops[i].type == 'Placeholder':
            for x in ops[i].outputs:
                print("output name = {}, shape: {},"
                      .format(x.name, x.get_shape())),

output name = input:0, shape: <unknown>,
output name = style_num:0, shape: (26,),


In [8]:
# 入力テンソルの名前を取得する
with tf.Graph().as_default() as g:
    tf.import_graph_def(gdef, name='')
    ops = g.get_operations()
    
    # プレースホルダの出力名を調べる
    N = len(ops)
    for i in range(N):
        if ops[i].type == 'Placeholder':
            for x in ops[i].outputs:
                print("output name = {}, shape: {},"
                      .format(x.name, x.get_shape())),

output name = input:0, shape: (1, 224, 224, 3),


In [14]:
with tf.Graph().as_default() as g:
    tf.import_graph_def(gdef, name='')
    ops = g.get_operations()
    
    # 最後の10ノードの入出力を調べる
    N = len(ops)
    for i in range(N-10,N):
        print('\n\nop id {} : op type: "{}"'.format(str(i), ops[i].type));
        print('input(s):'),
        for x in ops[i].inputs:
            print("name = {}, shape: {}, ".format(x.name, x.get_shape())),
        print('\noutput(s):'),
        for x in ops[i].outputs:
            print("name = {}, shape: {},".format(x.name, x.get_shape())),



op id 1454 : op type: "Add"
input(s):
name = transformer/expand/conv3/conv/InstanceNorm/batchnorm/mul_1:0, shape: (?, ?, ?, 3), 
name = transformer/expand/conv3/conv/InstanceNorm/batchnorm/sub:0, shape: (?, 1, 1, 3), 

output(s):
name = transformer/expand/conv3/conv/InstanceNorm/batchnorm/add_1:0, shape: (?, ?, ?, 3),


op id 1455 : op type: "Sigmoid"
input(s):
name = transformer/expand/conv3/conv/InstanceNorm/batchnorm/add_1:0, shape: (?, ?, ?, 3), 

output(s):
name = transformer/expand/conv3/conv/Sigmoid:0, shape: (?, ?, ?, 3),


op id 1456 : op type: "Const"
input(s):

output(s):
name = mul/y:0, shape: (),


op id 1457 : op type: "Mul"
input(s):
name = transformer/expand/conv3/conv/Sigmoid:0, shape: (?, ?, ?, 3), 
name = mul/y:0, shape: (), 

output(s):
name = mul:0, shape: (?, ?, ?, 3),


op id 1458 : op type: "Squeeze"
input(s):
name = mul:0, shape: (?, ?, ?, 3), 

output(s):
name = Squeeze:0, shape: <unknown>,


op id 1459 : op type: "Const"
input(s):

output(s):
name = strided

In [15]:
import tfcoreml
mlmodel = tfcoreml.convert(
    tf_model_path = tf_model_path,
    mlmodel_path = './stylize.mlmodel',
    output_feature_names = ['Squeeze:0'],
    image_input_names = ['input:0'])


Loading the TF graph...



Graph Loaded.


ValueError: Please provide the shape for the input input:0 through the argument 'input_name_shape_dict'

In [16]:
import tfcoreml

mlmodel = tfcoreml.convert(
    tf_model_path = tf_model_path,
    mlmodel_path = './stylize_1.mlmodel',
    output_feature_names = ['Squeeze:0'],
    input_name_shape_dict = {'input:0':[1,512,512,3], 'style_num:0':[26]},
    image_input_names = ['input:0'])


Loading the TF graph...
Graph Loaded.
Now finding ops in the TF graph that can be dropped for inference
Done.
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob style_num:0
Automatic shape interpretation succeeded for input blob input:0
1/1464: Analysing op name: strided_slice/stack_2 ( type:  Const )
2/1464: Analysing op name: strided_slice/stack_1 ( type:  Const )
3/1464: Analysing op name: strided_slice/stack ( type:  Const )
4/1464: Analysing op name: mul/y ( type:  Const )
5/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/ExpandDims_1/dim ( type:  Const )
6/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/ExpandDims/dim ( type:  Const )
7/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/Sum/reduction_indices ( type:  Const )
8/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/beta ( type:  Const )
9/1464: Analysing op name: transformer/expand/conv3/conv/Insta

834/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/Reshape_eightbit_quantize_style_num ( type:  QuantizeV2 )
835/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/Reshape_eightbit_quantized_reshape ( type:  QuantizedReshape )
836/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/Reshape ( type:  Dequantize )
837/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/mul ( type:  Mul )
838/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/Sum ( type:  Sum )
839/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/ExpandDims ( type:  ExpandDims )
840/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/ExpandDims_1 ( type:  ExpandDims )
841/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/Reshape_1_eightbit_reshape_style_num ( type:  Reshape )
842/1464: Analysing op name: transformer/contract/conv1/InstanceNorm/Reshape_1_eightbit_max_style_num ( type:  Max )
843/1464: Analysin

910/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/sufficient_statistics/count ( type:  Prod )
911/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/normalize/divisor ( type:  Reciprocal )
912/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/normalize/mean ( type:  Mul )
913/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/normalize/Square ( type:  Square )
914/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/sufficient_statistics/Square ( type:  Square )
915/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/sufficient_statistics/var_ss ( type:  Sum )
916/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/normalize/Mul ( type:  Mul )
917/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/moments/normalize/variance ( type:  Sub )
918/1464: Analysing op name: transformer/contract/conv2/InstanceNorm/batchnorm/ad

1012/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/sufficient_statistics/count ( type:  Prod )
1013/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/normalize/divisor ( type:  Reciprocal )
1014/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/normalize/mean ( type:  Mul )
1015/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/normalize/Square ( type:  Square )
1016/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/sufficient_statistics/Square ( type:  Square )
1017/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/sufficient_statistics/var_ss ( type:  Sum )
1018/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/normalize/Mul ( type:  Mul )
1019/1464: Analysing op name: transformer/residual/residual1/conv2/InstanceNorm/moments/normalize/variance ( type: 

1104/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/sufficient_statistics/count ( type:  Prod )
1105/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/normalize/divisor ( type:  Reciprocal )
1106/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/normalize/mean ( type:  Mul )
1107/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/normalize/Square ( type:  Square )
1108/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/sufficient_statistics/Square ( type:  Square )
1109/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/sufficient_statistics/var_ss ( type:  Sum )
1110/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/normalize/Mul ( type:  Mul )
1111/1464: Analysing op name: transformer/residual/residual3/conv1/InstanceNorm/moments/normalize/variance ( type: 

1201/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/sufficient_statistics/count ( type:  Prod )
1202/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/normalize/divisor ( type:  Reciprocal )
1203/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/normalize/mean ( type:  Mul )
1204/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/normalize/Square ( type:  Square )
1205/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/sufficient_statistics/Square ( type:  Square )
1206/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/sufficient_statistics/var_ss ( type:  Sum )
1207/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/normalize/Mul ( type:  Mul )
1208/1464: Analysing op name: transformer/residual/residual4/conv2/InstanceNorm/moments/normalize/variance ( type: 

1293/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/sufficient_statistics/count ( type:  Prod )
1294/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/normalize/divisor ( type:  Reciprocal )
1295/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/normalize/mean ( type:  Mul )
1296/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/normalize/Square ( type:  Square )
1297/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/sufficient_statistics/Square ( type:  Square )
1298/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/sufficient_statistics/var_ss ( type:  Sum )
1299/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/normalize/Mul ( type:  Mul )
1300/1464: Analysing op name: transformer/residual/residual6/conv1/InstanceNorm/moments/normalize/variance ( type: 

1363/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/sufficient_statistics/count ( type:  Prod )
1364/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/normalize/divisor ( type:  Reciprocal )
1365/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/normalize/mean ( type:  Mul )
1366/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/normalize/Square ( type:  Square )
1367/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/sufficient_statistics/Square ( type:  Square )
1368/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/sufficient_statistics/var_ss ( type:  Sum )
1369/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/normalize/Mul ( type:  Mul )
1370/1464: Analysing op name: transformer/expand/conv1/conv/InstanceNorm/moments/normalize/variance ( type:  Sub )
1371/1464: Analysing op name: transformer/expand/

1445/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/sufficient_statistics/count ( type:  Prod )
1446/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/normalize/divisor ( type:  Reciprocal )
1447/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/normalize/mean ( type:  Mul )
1448/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/normalize/Square ( type:  Square )
1449/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/sufficient_statistics/Square ( type:  Square )
1450/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/sufficient_statistics/var_ss ( type:  Sum )
1451/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/normalize/Mul ( type:  Mul )
1452/1464: Analysing op name: transformer/expand/conv3/conv/InstanceNorm/moments/normalize/variance ( type:  Sub )
1453/1464: Analysing op name: transformer/expand/

In [17]:
spec = mlmodel.get_spec()

In [18]:
print(spec.description)

input {
  name: "style_num__0"
  shortDescription: "This input is a sequence of length 26. Feed it an MLMultiArray of shape [26, 1, 1, 1, 1] at runtime"
  type {
    multiArrayType {
      shape: 1
      shape: 1
      shape: 1
      dataType: DOUBLE
    }
  }
}
input {
  name: "input__0"
  type {
    imageType {
      width: 512
      height: 512
      colorSpace: RGB
    }
  }
}
output {
  name: "Squeeze__0"
  type {
    multiArrayType {
      dataType: DOUBLE
    }
  }
}
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.3"
  }
}



In [20]:
output = spec.description.output[0]
print(output.name)

Squeeze__0


In [21]:
import coremltools

from coremltools.proto import FeatureTypes_pb2 as ft

output.type.imageType.colorSpace = ft.ImageFeatureType.ColorSpace.Value('RGB')
output.type.imageType.width = 512
output.type.imageType.height = 512

In [22]:
 print(spec.description)

input {
  name: "style_num__0"
  shortDescription: "This input is a sequence of length 26. Feed it an MLMultiArray of shape [26, 1, 1, 1, 1] at runtime"
  type {
    multiArrayType {
      shape: 1
      shape: 1
      shape: 1
      dataType: DOUBLE
    }
  }
}
input {
  name: "input__0"
  type {
    imageType {
      width: 512
      height: 512
      colorSpace: RGB
    }
  }
}
output {
  name: "Squeeze__0"
  type {
    imageType {
      width: 512
      height: 512
      colorSpace: RGB
    }
  }
}
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.3"
  }
}



In [23]:
coremltools.models.utils.save_spec(spec, './stylize.mlmodel')